In [ ]:
using Pkg
Pkg.activate(".")

#comment out after first installation
Pkg.add(name="PauliPropagation", version="0.3.0")
Pkg.add("ReverseDiff")
Pkg.add("OptimKit")

  Activating project at `~/Documents/aaETH/PhD/codes/cc2d_example`


In [13]:
using PauliPropagation
using ReverseDiff
using OptimKit

include("TFIM_example_utils.jl");

The goal is to compress the dynamics $U$ of $L_U$ layers of the TFIM Hamiltonian into an ansatz $V(\vec\theta)$ with $L_V$ layers ($L_V < L_U$) by minimizing
$$ R_{\mathcal{Q}_{LS}}^{\mathrm{loc}}(\vec \theta)= \frac{1}{2} - \frac{1}{6n} \sum_{j=1}^{n_q}\sum_{P = X, Y, Z} \langle\langle P_j \vert \boldsymbol{V(\vec\theta)U^\dagger}\vert P_j\rangle \rangle, $$
using `PauliPropagation.jl`.
This is an example notebook for the compression algorithm for 2D systems introduced in [[1]](https://arxiv.org/abs/2507.01883).

Lattice and couplings setup: create a rectangular topology of $n_x\times n_y$ qubits, specify the interaction strength and the transverse field for the TFIM Hamiltonian.

In [ ]:
nx = 3
ny = 3
nq = nx * ny
topo = rectangletopology(nx, ny)

J = 1.
h = 1.;

Specify the number of layers in the target $L_U$ and in the ansatz $L_V$.

In [15]:
LU = 8
LV = 2

t = 0.42;

Store the gates needed to implement a single layer of a second order trotterization $$ e^{-i \Delta t H_{TFIM}} \approx e^{-i \frac{\Delta t}2 H_{X}} e^{-i \Delta t H_{ZZ}} e^{-i \frac{\Delta t}2 H_{X}}$$
Note that `PauliRotation(theta)` gates implement a rotation with angle `theta/2`, so we need to multiply all our angles by a factor two.

In [ ]:
single_layer::Vector{Gate} = []
thetaU_single_layer::Vector{Float64} = []

dt = t / LU

for i in 1:nq
    push!(single_layer, PauliRotation(:X, i))
    push!(thetaU_single_layer, dt * h)
end

for pair in topo
    push!(single_layer, PauliRotation([:Z, :Z], pair))
    push!(thetaU_single_layer, dt * J * 2)
end

for i in 1:nq
    push!(single_layer, PauliRotation(:X, i))
    push!(thetaU_single_layer, dt * h)
end

circU = repeat(single_layer, LU)
thetasU = repeat(thetaU_single_layer, LU)
CoeffType = eltype(thetasU)

Float64

Specify PP truncations.

In [17]:
min_abs_coeff_U = 1.e-10
min_abs_coeff_V = 1.e-11
max_sins_U = 11
max_sins_V = 12
W = 8;

Create a list `all_symbs_U` containing all $3n_q$ Pauli strings that we need to compute the cost function

In [ ]:
iter_paulis = []
for site = 1:nq
    for sigma in [:X, :Y, :Z]
        push!(iter_paulis, (site, sigma))
    end
end
sigma_to_index = Dict(:X => 1, :Y => 2, :Z => 3)

all_symbs_U = []
for (site, sigma) in iter_paulis
    symb = PauliSum(nq, CoeffType)
    add!(symb, sigma, site, CoeffType(1.0))
    push!(all_symbs_U, wrapcoefficients(symb, PauliFreqTracker))
end

Compute $\bm{U^\dagger}\vert P_j\rangle\rangle$ for all strings in `all_symbs_U`

In [19]:
for index = 1:size(all_symbs_U)[1]
    propagate!(reverse(circU), all_symbs_U[index], -reverse(thetasU),
        min_abs_coeff=min_abs_coeff_U, max_sins=max_sins_U, max_weight=W)
end

Here we compute $\langle \langle P_j\vert \boldsymbol{V(\vec\theta)}$ and then evaluate the inner product $\langle \langle P_j\vert \boldsymbol{V(\vec\theta)} \cdot \boldsymbol{U^\dagger}\vert P_j\rangle\rangle$.
Since we are interested in optimizing over $\vec\theta$, we need to extract the gradient as well. To this end we emply the `ReverseDiff` AD library, which allows us to "prerecord" a compiled tape of the gradient that we can use for the later optimization. For each Pauli string we precompile the tape of $\langle \langle P_j\vert \boldsymbol{V(\vec\theta)} \cdot \boldsymbol{U^\dagger}\vert P_j\rangle\rangle$, and we store all of them in `tapes`.

In [20]:
circV = repeat(single_layer, LV)

tapes = create_tapes(circV, iter_paulis, W, max_sins_V, min_abs_coeff_V, all_symbs_U, ones(length(circV)), nq)

fg = function (params)
    return Cloc(iter_paulis, params, nq, tapes; return_grad=true)
end

f = function (params)
    return Cloc(iter_paulis, params, nq, tapes; return_grad=false)
end;

Perform the optimization starting from the initial guess `x_trotter` corresponding to the Trotter angles for $L_V$ layers.

In [ ]:
gradtol = 1.e-12
maxiter = 20
cg = OptimKit.ConjugateGradient(verbosity=1, gradtol=gradtol, maxiter=maxiter)

x_trotter = repeat(thetaU_single_layer, LV) .* (LU / LV)
x, fx, gx, numfg, normgradhistory = OptimKit.optimize(fg, x_trotter, cg);

┌ Warning: CG: not converged to requested tol after 20 iterations and time 4.44 s: f = 0.000078202419, ‖∇f‖ = 1.5452e-05
└ @ OptimKit /Users/Matteo/.julia/packages/OptimKit/G6i79/src/cg.jl:172


In [22]:
println("Improvement over Trotter: $(f(x_trotter) / fx)")

Improvement over Trotter: 6.244404100079742
